In [ ]:
library(keras)
library(reticulate)
library(ggplot2)
library(gridExtra)
use_condaenv("r-tensorflow")
use_session_with_seed(7)
options(keras.view_metrics = TRUE)

# 2.2 - Using convnets with small datasets

This notebook contains the code samples found in Chapter 5, Section 2 of [Deep Learning with R](https://www.manning.com/books/deep-learning-with-r). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

## Training a convnet from scratch on a small dataset

Having to train an image classification model using only very little data is a common situation, which you likely encounter yourself in 
practice if you ever do computer vision in a professional context.

Having "few" samples can mean anywhere from a few hundreds to a few tens of thousands of images. As a practical example, we will focus on 
classifying images as "dogs" or "cats", in a dataset containing 4000 pictures of cats and dogs (2000 cats, 2000 dogs). We will use 2000 
pictures for training, 1000 for validation, and finally 1000 for testing.

In this section, we will review one basic strategy to tackle this problem: training a new model from scratch on what little data we have. We 
will start by naively training a small convnet on our 2000 training samples, without any regularization, to set a baseline for what can be 
achieved. This will get us to a classification accuracy of 71%. At that point, our main issue will be overfitting. Then we will introduce 
*data augmentation*, a powerful technique for mitigating overfitting in computer vision. By leveraging data augmentation, we will improve 
our network to reach an accuracy of 82%.

In the next section, we will review two more essential techniques for applying deep learning to small datasets: *doing feature extraction 
with a pre-trained network* (this will get us to an accuracy of 90% to 93%), and *fine-tuning a pre-trained network* (this will get us to 
our final accuracy of 95%). Together, these three strategies -- training a small model from scratch, doing feature extracting using a 
pre-trained model, and fine-tuning a pre-trained model -- will constitute your future toolbox for tackling the problem of doing computer 
vision with small datasets.

## The relevance of deep learning for small-data problems

You will sometimes hear that deep learning only works when lots of data is available. This is in part a valid point: one fundamental 
characteristic of deep learning is that it is able to find interesting features in the training data on its own, without any need for manual 
feature engineering, and this can only be achieved when lots of training examples are available. This is especially true for problems where 
the input samples are very high-dimensional, like images.

However, what constitutes "lots" of samples is relative -- relative to the size and depth of the network you are trying to train, for 
starters. It isn't possible to train a convnet to solve a complex problem with just a few tens of samples, but a few hundreds can 
potentially suffice if the model is small and well-regularized and if the task is simple. 
Because convnets learn local, translation-invariant features, they are very 
data-efficient on perceptual problems. Training a convnet from scratch on a very small image dataset will still yield reasonable results 
despite a relative lack of data, without the need for any custom feature engineering. You will see this in action in this section.

But what's more, deep learning models are by nature highly repurposable: you can take, say, an image classification or speech-to-text model 
trained on a large-scale dataset then reuse it on a significantly different problem with only minor changes. Specifically, in the case of 
computer vision, many pre-trained models (usually trained on the ImageNet dataset) are now publicly available for download and can be used 
to bootstrap powerful vision models out of very little data. That's what we will do in the next section.

For now, let's get started by getting our hands on the data.

## Downloading the data

The cats vs. dogs dataset that we will use isn't packaged with Keras. It was made available by Kaggle.com as part of a computer vision competition in late 2013, back when convnets weren't quite mainstream. 

The pictures are medium-resolution color JPEGs. They look like this:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

Unsurprisingly, the cats vs. dogs Kaggle competition in 2013 was won by entrants who used convnets. 
The best entries could achieve up to 95% accuracy. In our own example, we will get fairly close to this accuracy (in the next section), even though we will be training our models on less than 10% of the data that was available to the competitors.


This original dataset contains 25,000 images of dogs and cats (12,500 from each class) and is 543MB large (compressed). You can download it at: `https://www.kaggle.com/c/dogs-vs-cats/data` (you will need to create a Kaggle account if you don't already have one -- don't worry, the process is painless). 

For this notebook we have already prepared the data by splititng it into three subsets: a training set with 1000 samples of each class, a validation set with 500 samples of each class, and finally a test set with 500 samples of each class.

**Please download the prepared [data](https://drive.google.com/open?id=1H5do852RlW7EdALFQOiP6hro7zB--AY9) and unzip it within or move to the notebooks folder**

Here are a few lines of code to do this:

In [ ]:
# Setting up the file names
# Note we already copied the required data into the relevant folders.

base_dir <- "../data/cats_and_dogs_small"

train_dir <- file.path(base_dir, "train")
validation_dir <- file.path(base_dir, "validation")
test_dir <- file.path(base_dir, "test")

train_cats_dir <- file.path(train_dir, "cats")

train_dogs_dir <- file.path(train_dir, "dogs")

validation_cats_dir <- file.path(validation_dir, "cats")

validation_dogs_dir <- file.path(validation_dir, "dogs")

test_cats_dir <- file.path(test_dir, "cats")

test_dogs_dir <- file.path(test_dir, "dogs")

As a sanity check, let's count how many pictures we have in each training split (train/validation/test):

In [ ]:
cat("total training cat images:", length(list.files(train_cats_dir)))

In [ ]:
cat("total training dog images:", length(list.files(train_dogs_dir)))

In [ ]:
cat("total validation cat images:", length(list.files(validation_cats_dir)))

In [ ]:
cat("total validation dog images:", length(list.files(validation_dogs_dir)))

In [ ]:
cat("total test cat images:", length(list.files(test_cats_dir)))

In [ ]:
cat("total test dog images:", length(list.files(test_dogs_dir)))


So we have indeed 2000 training images, and then 1000 validation images and 1000 test images. In each split, there is the same number of 
samples from each class: this is a balanced binary classification problem, which means that classification accuracy will be an appropriate 
measure of success.

## Building our network

We've already built a small convnet for MNIST in the previous example, so you should be familiar with them. We will reuse the same 
general structure: our convnet will be a stack of alternated `Conv2D` (with `relu` activation) and `MaxPooling2D` layers.

However, since we are dealing with bigger images and a more complex problem, we will make our network accordingly larger: it will have one 
more `Conv2D` + `MaxPooling2D` stage. This serves both to augment the capacity of the network, and to further reduce the size of the 
feature maps, so that they aren't overly large when we reach the `Flatten` layer. Here, since we start from inputs of size 150x150 (a 
somewhat arbitrary choice), we end up with feature maps of size 7x7 right before the `Flatten` layer.

Note that the depth of the feature maps is progressively increasing in the network (from 32 to 128), while the size of the feature maps is 
decreasing (from 148x148 to 7x7). This is a pattern that you will see in almost all convnets.

Since we are attacking a binary classification problem, we are ending the network with a single unit (a `Dense` layer of size 1) and a 
`sigmoid` activation. This unit will encode the probability that the network is looking at one class or the other.

In [ ]:
# library(keras)

model <- keras_model_sequential() %>% 
  layer_conv_2d(filters = 32, kernel_size = c(3, 3), activation = "relu",
                input_shape = c(150, 150, 3)) %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 64, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 128, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 128, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_flatten() %>% 
  layer_dense(units = 512, activation = "relu") %>% 
  layer_dense(units = 1, activation = "sigmoid")


Let's take a look at how the dimensions of the feature maps change with every successive layer:

In [ ]:
summary(model)

For our compilation step, we'll go with the `RMSprop` optimizer as usual. Since we ended our network with a single sigmoid unit, we will use binary crossentropy as our loss.

| Problem type        | Last-layer activation           | Loss function |
| :------------ |:-------------:| -----:|
| Binary classification      | sigmoid | binary_crossentropy |
| Multiclass, single-label classification      | softmax      |   categorical_crossentropy |
| Multiclass, multilabel classification | sigmoid      |    binary_crossentropy |
| Regression to arbitary values | None | mse |
| Regression to values between 0 and 1 | sigmoid | mse or binary_crossentropy |

In [ ]:
model %>% compile(
  loss = "binary_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-4),
  metrics = c("acc")
)

## Data preprocessing

As you already know by now, data should be formatted into appropriately pre-processed floating point tensors before being fed into our 
network. Currently, our data sits on a drive as JPEG files, so the steps for getting it into our network are roughly:

1. Read the picture files.
2. Decode the JPEG content to RBG grids of pixels.
3. Convert these into floating point tensors.
4. Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values).

It may seem a bit daunting, but thankfully Keras has utilities to take care of these steps automatically. Keras includes a number of image processing helper tools. In particular, it includes the `image_data_generator()` function, which can automatically turn image files on disk into batches of pre-processed tensors. This is what we will use here.


In [ ]:
# All images will be rescaled by 1/255
train_datagen <- 
validation_datagen <- 

train_generator <- flow_images_from_directory(
# This is the target directory (training data)

# This is the data generator

# All images will be resized to 150x150
target_size = c(150, 150),
# Batch size

# Since we use binary_crossentropy loss, we need binary labels
)

validation_generator <- flow_images_from_directory(
#equivalent to the above but for the validation directory
)

Let's look at the output of one of these generators: it yields batches of 150 × 150 RGB images (shape `(20, 150, 150, 3)`) and binary labels (shape `(20)`). There are 20 samples in each batch (the batch size). Note that the generator yields these batches indefinitely: it loops endlessly over the images in the target folder.

In [ ]:
batch <- generator_next(train_generator)
str(batch)

Let's fit our model to the data using the generator. We do it using the `fit_generator` method, the equivalent of `fit` for data generators 
like ours. It expects as first argument a Python generator that will yield batches of inputs and targets indefinitely, like ours does. 
Because the data is being generated endlessly, the generator needs to know example how many samples to draw from the generator before 
declaring an epoch over. This is the role of the `steps_per_epoch` argument: after having drawn `steps_per_epoch` batches from the 
generator, i.e. after having run for `steps_per_epoch` gradient descent steps, the fitting process will go to the next epoch. In our case, 
batches are 20-sample large, so it will take 100 batches until we see our target of 2000 samples.

When using `fit_generator`, one may pass a `validation_data` argument, much like with the `fit` method. Importantly, this argument is 
allowed to be a data generator itself, but it could be a tuple of Numpy arrays as well. If you pass a generator as `validation_data`, then 
this generator is expected to yield batches of validation data endlessly, and thus you should also specify the `validation_steps` argument, 
which tells the process how many batches to draw from the validation generator for evaluation.

In [ ]:
# history <- model %>% fit_generator(
#   train_generator,
#   steps_per_epoch = 100,
#   epochs = 2,
#   validation_data = validation_generator,
#   validation_steps = 50
# )

# It is good practice to always save your models after training:
# model %>% save_model_hdf5("cats_and_dogs_small_1.h5")

<center><h2> As before, we don't have time to train the model, so we load a pretrained model instead

In [ ]:
model <- load_model_hdf5("../data/models/cats_and_dogs_small_1.h5")
history  <- py_load_object('../data/models/2-2-GPU-history.pk')
df <- data.frame(val_loss=unlist(history$val_loss), val_acc=unlist(history$val_acc), loss=unlist(history$loss), acc=unlist(history$acc), epochs=seq(length(history$val_loss)))
summary(model)

Let's plot the loss and accuracy of the model over the training and validation data during training:

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

p1 <- ggplot(df, aes(x=epochs)) +
  geom_point(aes( y=loss, colour = "Trainig loss")) +
  geom_line(aes(y=val_loss,colour = "Validation loss")) +
  scale_colour_manual("",values=c("blue","orange"))
p2 <- ggplot(df, aes(x=epochs)) +
  geom_point(aes( y=acc, colour = "Training acc")) +
  geom_line(aes(y=val_acc,colour = "Validation acc")) +
  scale_colour_manual("",values=c("blue","orange"))

grid.arrange(p1,p2)

**These plots are characteristic of overfitting**. 
Our training accuracy increases linearly over time, until it reaches nearly 100%, while our 
validation accuracy stalls at 70-72%. Our validation loss reaches its minimum after only five epochs then stalls, while the training loss keeps decreasing linearly until it reaches nearly 0.

Because we only have **relatively few training samples** (2000), **overfitting** is going to be our number one concern. 
A technique that can help mitigate overfitting specific to computer vision, and which is used almost universally when processing images with deep learning models, is: **data augmentation**.

## Using data augmentation

Overfitting is caused by having too few samples to learn from, rendering us unable to train a model able to generalize to new data. Given infinite data, our model would be exposed to every possible aspect of the data distribution at hand: we would never overfit. Data augmentation takes the approach of generating more training data from existing training samples, by "augmenting" the samples via a number of random transformations that yield believable-looking images. The goal is that at training time, our model would never see the exact same picture twice. This helps the model get exposed to more aspects of the data and generalize better.

In Keras, this can be done by configuring a number of random transformations to be performed on the images read by an `image_data_generator()`. 

There are several options available (for more, see the Keras documentation). Here we'll use:

* `rotation_range` is a value in degrees (0-180), a range within which to randomly rotate pictures.
* `width_shift` and `height_shift` are ranges (as a fraction of total width or height) within which to randomly translate pictures 
vertically or horizontally.
* `shear_range` is for randomly applying shearing transformations.
* `zoom_range` is for randomly zooming inside pictures.
* `horizontal_flip` is for randomly flipping half of the images horizontally -- relevant when there are no assumptions of horizontal 
asymmetry (e.g. real-world pictures).
* `fill_mode` is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

In [ ]:
# Let's fill in the above arguments
datagen <- image_data_generator(
)


Let's take a look at our augmented images:

In [ ]:
# We pick one image to "augment"
fnames <- list.files(train_cats_dir, full.names = TRUE)
img_path <- fnames[[4]]

# Convert it to an array with shape (150, 150, 3)
img <- image_load(img_path, target_size = c(150, 150))
img_array <- image_to_array(img)
img_array <- array_reshape(img_array, c(1, 150, 150, 3))

# Generated that will flow augmented images
augmentation_generator <- flow_images_from_data(
  img_array, 
  generator = datagen, 
  batch_size = 1 
)

# Plot the first 4 augmented images
op <- par(mfrow = c(2, 2), pty = "s", mar = c(1, 0, 1, 0))
for (i in 1:4) {
  batch <- generator_next(augmentation_generator)
  plot(as.raster(batch[1,,,]))
}
par(op)

If we train a new network using this data augmentation configuration, our network will never see twice the same input. However, the inputs that it sees are still heavily intercorrelated, since they come from a small number of original images -- we cannot produce new information, we can only remix existing information. 
As such, this might not be quite enough to completely get rid of overfitting. 

To further fight overfitting, we will also add a dropout layer to our model, right before the densely-connected classifier:

In [ ]:
model <- keras_model_sequential() %>% 
  layer_conv_2d(filters = 32, kernel_size = c(3, 3), activation = "relu",
                input_shape = c(150, 150, 3)) %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 64, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 128, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_conv_2d(filters = 128, kernel_size = c(3, 3), activation = "relu") %>% 
  layer_max_pooling_2d(pool_size = c(2, 2)) %>% 
  layer_flatten() %>% 
  layer_dropout(rate = 0.5) %>% 
  layer_dense(units = 512, activation = "relu") %>% 
  layer_dense(units = 1, activation = "sigmoid")  
  
model %>% compile(
  loss = "binary_crossentropy",
  optimizer = optimizer_rmsprop(lr = 1e-4),
  metrics = c("acc")
)

summary(model)

<center><h3>Let's train our network using data augmentation and dropout:

In [ ]:
datagen <- image_data_generator(
 # as above, plus we add the image rescale
  rescale = 1/255,
  rotation_range = 40,
  width_shift_range = 0.2,
  height_shift_range = 0.2,
  shear_range = 0.2,
  zoom_range = 0.2,
  horizontal_flip = TRUE
)

# Note that the validation data should not be augmented!
# we will simply rescale it
test_datagen <- image_data_generator(rescale = 1/255)

train_generator <- flow_images_from_directory(
  train_dir,
  datagen,
  target_size = c(150, 150),
  batch_size = 32,
  class_mode = "binary"
)

validation_generator <- flow_images_from_directory(
  validation_dir,
  test_datagen,
  target_size = c(150, 150),
  batch_size = 32,
  class_mode = "binary"
)

# history <- model %>% fit_generator(
#   train_generator,
#   steps_per_epoch = 100,
#   epochs = 2,
#   validation_data = validation_generator,
#   validation_steps = 50
# )
# # Let's save our model -- we will be using it in the section on convnet visualization.
# model %>% save_model_hdf5("cats_and_dogs_small_2.h5")

<center><h2> As before, we don't have time to train the model, so we load a pretrained model instead

In [ ]:
model <- load_model_hdf5("../data/models/cats_and_dogs_small_2.h5")
history  <- py_load_object('../data/models/2-2-GPU-history_2.pk')
df <- data.frame(val_loss=unlist(history$val_loss), val_acc=unlist(history$val_acc), loss=unlist(history$loss), acc=unlist(history$acc), epochs=seq(length(history$val_loss)))
summary(model)

Let's plot our results again:

In [ ]:
options(repr.plot.width=8, repr.plot.height=6)

p1 <- ggplot(df, aes(x=epochs)) +
  geom_point(aes( y=loss, colour = "Training loss")) +
  geom_line(aes(y=val_loss,colour = "Validation loss")) +
  scale_colour_manual("",values=c("blue","orange"))
p2 <- ggplot(df, aes(x=epochs)) +
  geom_point(aes( y=acc, colour = "Training acc")) +
  geom_line(aes(y=val_acc,colour = "Validation acc")) +
  scale_colour_manual("",values=c("blue","orange"))

grid.arrange(p1,p2)

Thanks to data augmentation and dropout, we are no longer overfitting: the training curves are rather closely tracking the validation 
curves. We are now able to reach an accuracy of 82%, a 15% relative improvement over the non-regularized model.

By leveraging regularization techniques even further and by tuning the network's parameters (such as the number of filters per convolution 
layer, or the number of layers in the network), we may be able to get an even better accuracy, likely up to 86-87%. However, it would prove 
very difficult to go any higher just by training our own convnet from scratch, simply because we have so little data to work with. As a 
next step to improve our accuracy on this problem, we will have to leverage a pre-trained model, which will be the focus of the next two 
sections.

## Let's Compare the two and recap¶


In [ ]:
before_history  <- py_load_object('../data/models/2-2-GPU-history.pk')
df_before <- data.frame(val_loss=unlist(before_history$val_loss), val_acc=unlist(before_history$val_acc), 
                 loss=unlist(before_history$loss), acc=unlist(before_history$acc), 
                 epochs=seq(length(before_history$val_loss)))


options(repr.plot.width=8, repr.plot.height=6)

p1 <- ggplot(df_before, aes(x=epochs)) +
  geom_point(aes( y=acc, colour = "Training acc")) +
  geom_line(aes(y=val_acc,colour = "Validation acc")) +
  scale_colour_manual("",values=c("blue","orange")) +
  ggtitle("Before Data Augmentation")  + theme_bw()

p2 <- ggplot(df, aes(x=epochs)) +
  geom_point(aes( y=acc, colour = "Training acc")) +
  geom_line(aes(y=val_acc,colour = "Validation acc")) +
  scale_colour_manual("",values=c("blue","orange")) +
  ggtitle("After Data Augmentation") + theme_bw()

grid.arrange(p1,p2)